# Lektion 9

In [ ]:
from sympy import *
init_printing()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

## Funktionen als Argumente

"Functions are first class citizens"

In [ ]:
def komposition(f, g):
    "gibt die Funktion f ∘ g zurück"
    def func(x):
        return f(g(x))
    return func

In [ ]:
h = komposition(cos, sqrt)
h(pi**2)

In [ ]:
x = Symbol('x')
h(x)

## Jordansche Normalform

In [ ]:
C = Matrix(3, 3, [-4, -2, -3, 5, 3, 3, 5, 2, 4])
C

In [ ]:
C.eigenvals()

In [ ]:
C.eigenvects()

Das ist einer zu wenig.

In [ ]:
# T, J = C.diagonalize()  # MatrixError

In [ ]:
T, J = C.jordan_form()
T, J

In [ ]:
T * J * T**(-1) == C

## Rang einer Matrix

In [ ]:
M = Matrix(3, 3, range(1,10))
M

In [ ]:
M.rank()

In [ ]:
x = Symbol('x')
y = Symbol('y')
M = Matrix(3, 2, [2*x+2, 2*y-2, 2*x+2, -2*y+2, y-1, x+1])
M

In [ ]:
M.rank()

Glauben wir das?

In [ ]:
def mult_zeile(A, zeilennr, faktor):
    B = Matrix(A)
    def func(b, j):
        return b*faktor
    B.row_op(zeilennr, func)
    return B

def add_zeile(A, quell_nr, ziel_nr, faktor):
    B = Matrix(A)  
    def func(b, j):
        return b + B[quell_nr,j]*faktor
    B.row_op(ziel_nr, func)
    return B

In [ ]:
M1 = add_zeile(M, 0, 1, -1)
M1

In [ ]:
M2 = mult_zeile(M1, 0, y-1)   # nur für y != 1
M3 = mult_zeile(M2, 2, x+1)   # nur für x != -1
M3

In [ ]:
M4 = add_zeile(M3, 0, 2, -Rational(1,2)).expand()
factor(M4)

Wenn $x\ne-1$ und $y\ne1$, dann ist der Rang dieser Matrix gleich 2

In [ ]:
M5 = M.subs(x, -1)
factor(M5)

Für $y\ne1$ ist der Rang dieser Matrix immer noch 2.

In [ ]:
M6 = M.subs(y, 1)
factor(M6)

dito für $x\ne-1$

In [ ]:
M7 = M.subs({x:-1, y:1})
M7

Kein Fehler von `sympy`.

## Normen von Vektoren und Matrizen

In [ ]:
v = Matrix([1,2,3])
v

In [ ]:
v.norm()

In [ ]:
sqrt((v.T*v)[0])

In [ ]:
v.norm(oo)

In [ ]:
v.norm(1)

In [ ]:
C.norm()

Die Frobeniusnorm ist submultiplikativ, also $\left\Vert A B \right\Vert \le \left\Vert A \right\Vert \left\Vert B \right\Vert$.  Sie ist aber keine Matrixnorm im Sinne der Analysis II. 

In [ ]:
C.norm(2)

In [ ]:
(C.T*C).eigenvals()

In [ ]:
C.norm().n(), C.norm(2).n()

## Kreuzprodukt

In [ ]:
w = Matrix([-1,-2,3])
z = v.cross(w)
z

In [ ]:
v.T*z

In [ ]:
w.T*z

## Gradient

In [ ]:
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')
var = [x,y,z]

In [ ]:
f = exp(x**2 + 2*y**2 + 3*z**2)
f

In [ ]:
gr = Matrix([f.diff(t) for t in var])
gr

sympy hat eine Funktion gradient, aber nur in 3D.  Man beachte aber den nächsten Abschnitt.

## Jacobi- und Hessematrix

In [ ]:
J = Matrix([f]).jacobian(var)
J

In [ ]:
H = factor(gr.jacobian(var))
H

Das ist die Hesse Matrix.

In [ ]:
factor(hessian(f, var)) == H

In [ ]:
f = Function('f')
f(x,y,z)

In [ ]:
hessian(f(x,y,z), var)

## Definitheit

In [ ]:
H1 = H.subs({x:1, y:0, z:-1})
H1

In [ ]:
H1.is_positive

In [ ]:
H1.det()

In [ ]:
for tmp in H1.eigenvals():
    display((tmp, tmp.n()))

In [ ]:
ImmutableMatrix(eye(3)).is_positive

Hurwitz Kriterium

In [ ]:
for j in range(1,4):
    minor = H1[0:j, 0:j]
    display(minor.det())

Also positiv definit

In [ ]:
for j in range(1,4):
    minor = -H1[0:j, 0:j]
    display(minor.det())

Negativ definit.  

Indefinit genau dann, wenn eine Unterdeterminante gerader Ordnung negativ ist.

## Extremwerte in mehreren Veränderlichen

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
f = -x**4/2 - x**2*y**2 - y**4/2 + x**3 - 3*x*y**2
fn = lambdify((x,y), f)
f

In [ ]:
xn = np.linspace(-5, 5)
yn = np.linspace(-5, 5)
X, Y = np.meshgrid(xn, yn)
W = fn(X, Y)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, W, rstride=1, cstride=1, linewidth=0,
                cmap=plt.cm.coolwarm);

In [ ]:
glg1 = Eq(f.diff(x), 0)
glg2 = Eq(f.diff(y), 0)
gls = {glg1, glg2}
gls

In [ ]:
Lsg = solve(gls)
Lsg

In [ ]:
H = hessian(f, [x,y])
H

In [ ]:
H1 = H.subs(Lsg[0])
H1

In [ ]:
H1[0,0]

In [ ]:
H1.det()

negativ definit

In [ ]:
H2 = H.subs(Lsg[1])
H2

In [ ]:
H2.det()

In [ ]:
H3 = H.subs(Lsg[2])
H3

In [ ]:
H4 = H.subs(Lsg[3])
H4

In [ ]:
for l in Lsg:
    y0 = f.subs(l)
    display((y0, y0.n()))

In [ ]:
from matplotlib.colors import Normalize
norm = Normalize(-.5, 1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, W, rstride=1, cstride=1, linewidth=0,
                cmap=plt.cm.coolwarm, norm=norm);

In [ ]:
for l in Lsg:
    display((x.subs(l).n(), y.subs(l).n()))

In [ ]:
xn = np.linspace(-1.5, 2, 100)
yn = np.linspace(-1.8, 1.8, 100)
X, Y = np.meshgrid(xn, yn)
W = np.maximum(fn(X, Y), -.1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, W, rstride=1, cstride=1, linewidth=0,
                cmap=plt.cm.coolwarm)

In [ ]:
f.subs(y, 0)

Also Sattel in $(0,0)$.